In [2]:
from bitarray import bitarray
import random as random

# Permutation and translation tables for DES
pc1 = [56, 48, 40, 32, 24, 16,  8,
  0, 57, 49, 41, 33, 25, 17,
  9,  1, 58, 50, 42, 34, 26,
  18, 10,  2, 59, 51, 43, 35,
  62, 54, 46, 38, 30, 22, 14,
  6, 61, 53, 45, 37, 29, 21,
  13,  5, 60, 52, 44, 36, 28,
  20, 12,  4, 27, 19, 11,  3]

# number left rotations of pc1
rotations = [ 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1]

# permuted choice key (table 2)
pc2 = [ 13, 16, 10, 23,  0,  4,
  2, 27, 14,  5, 20,  9,
  22, 18, 11,  3, 25,  7,
  15,  6, 26, 19, 12,  1,
  40, 51, 30, 36, 46, 54,
  29, 39, 50, 44, 32, 47,
  43, 48, 38, 55, 33, 52,
  45, 41, 49, 35, 28, 31]

# initial permutation IP
ip = [57, 49, 41, 33, 25, 17, 9,  1,
  59, 51, 43, 35, 27, 19, 11, 3,
  61, 53, 45, 37, 29, 21, 13, 5,
  63, 55, 47, 39, 31, 23, 15, 7,
  56, 48, 40, 32, 24, 16, 8,  0,
  58, 50, 42, 34, 26, 18, 10, 2,
  60, 52, 44, 36, 28, 20, 12, 4,
  62, 54, 46, 38, 30, 22, 14, 6]

# Expansion table for turning 32 bit blocks into 48 bits
expansion_table = [31,  0,  1,  2,  3,  4,
  3,  4,  5,  6,  7,  8,
  7,  8,  9, 10, 11, 12,
  11, 12, 13, 14, 15, 16,
  15, 16, 17, 18, 19, 20,
  19, 20, 21, 22, 23, 24,
  23, 24, 25, 26, 27, 28,
  27, 28, 29, 30, 31,  0]
    
sbox = [# S1
  [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7,
  0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8,
  4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0,
  15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13],

  # S2
  [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10,
  3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5,
  0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15,
  13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9],

  # S3
  [10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8,
  13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1,
  13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7,
  1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12],

  # S4
  [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15,
  13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9,
  10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4,
  3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14],

  # S5
  [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9,
  14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6,
  4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14,
  11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3],

  # S6
  [12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11,
  10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8,
  9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6,
  4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13],

  # S7
  [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1,
  13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6,
  1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2,
  6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12],

  # S8
  [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7,
  1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2,
  7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8,
  2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11],
]

# 32-bit permutation function P used on the output of the S-boxes
p = [
  15, 6, 19, 20, 28, 11,
  27, 16, 0, 14, 22, 25,
  4, 17, 30, 9, 1, 7,
  23,13, 31, 26, 2, 8,
  18, 12, 29, 5, 21, 10,
  3, 24]

# final permutation IP^-1
fp = [
  39,  7, 47, 15, 55, 23, 63, 31,
  38,  6, 46, 14, 54, 22, 62, 30,
  37,  5, 45, 13, 53, 21, 61, 29,
  36,  4, 44, 12, 52, 20, 60, 28,
  35,  3, 43, 11, 51, 19, 59, 27,
  34,  2, 42, 10, 50, 18, 58, 26,
  33,  1, 41,  9, 49, 17, 57, 25,
  32,  0, 40,  8, 48, 16, 56, 24]


def bitfield(n):
    return [int(digit) for digit in bin(n)[2:]]
    
def readBits(filename):
    reader = open(filename, 'rb')
    bits = bitarray()
    bits.fromfile(reader)
    return bits

def permutation(table, block):
    return list(map(lambda x: block[x], table))

def generateKey():
    return bitarray([random.randint(0, 1) for i in xrange(64)])

def generateSessionKeys(key):
    p1Key = permutation(pc1, key)
    leftKey = p1Key[:28]
    rightKey = p1Key[28:]
    keys = []
    for i in xrange(16):
        tempKey = bitarray()
        j = 0
        while (j < rotations[i]):
            leftKey.append(leftKey[0])
            leftKey.pop(0)
            rightKey.append(rightKey[0])
            rightKey.pop(0)
            j += 1
        tempKey = bitarray(leftKey)
        tempKey.extend(rightKey)
        keys.append(permutation(pc2,  tempKey))
    return keys  

    
def des(block, key, mode):
    keys = generateSessionKeys(key)
    if mode == 1:
        keys.reverse()
    ipBlock = permutation(ip, block)
    leftBlock = ipBlock[:32]
    rightBlock = ipBlock[32:]
    tempR = rightBlock[:]
    i = 0
    while i < 16:
        tempR = rightBlock[:]
        rightBlock = permutation(expansion_table, rightBlock)
        rightBlock = bitarray(map(lambda x,y: x^y, rightBlock, keys[i]))
        B = [rightBlock[:6], rightBlock[6:12], rightBlock[12:18], rightBlock[18:24], rightBlock[24:30], rightBlock[30:36],
             rightBlock[36:42], rightBlock[42:48]]
        f = bitarray()
    
        for j in xrange(8):
            q = bitarray()
            r = 2 * B[j][0]  + B[j][5]
            s = 8 * B[j][1] + 4 * B[j][2] + 2 * B[j][3] + B[j][4]
            l = bitarray(bitfield(sbox[j][r*16 + s]))
            if l.length() < 4:
                q = bitarray((4-l.length())*[0])
            q.extend(l)
            f.extend(q)
        rightBlock = permutation(p,f)
        rightBlock = bitarray(map(lambda x,y: x^y, rightBlock, leftBlock))
        leftBlock = tempR
        i += 1
    rightBlock.extend(leftBlock)
    final = permutation(fp, rightBlock)
    return final

def tripleDes(block, keys, mode):
    if mode == 0:
        block = des(block, keys[0], 0)
        block = des(block, keys[1], 0)
        block = des(block, keys[2], 0)
    else:
        block = des(block, keys[2], 1)
        block = des(block, keys[1], 1)
        block = des(block, keys[0], 1)
    return block

def encrypt(data, keys, mode):
    
    size = data.length() / 64
    adding = data.length() % 64
    encryptedData = bitarray()
    for i in xrange(size):
        index = i*64
        encryptedData.extend(tripleDes(data[index:index + 64], keys, mode))
    encryptedData.extend(data[64*size:])
    return encryptedData

def hash_function(data, keys, mode):
    l = bitarray(bitfield(len(data)))
    new_keys = keys
    size = data.length() / 64
    adding = data.length() % 64
    temp = 0
    for i in xrange(size):
        if temp == 3:
            temp = 0
        index = i * 64
        block = data[index:index + 64]
        new_block = tripleDes(block, new_keys, mode)
        new_keys[temp] = bitarray(map(lambda x,y: x^y, new_block, block))
        temp += 1
    last = data[64*size:]
    last.extend(bitarray([0] * (64 - adding)))
    last = bitarray(tripleDes(last, new_keys, mode))
    print(len(last))
    #encryptedData.extend(l)
    return int(last.to01(),2)
    

data = readBits("data.txt")
keys = []
for i in xrange(3):
    keys.append(generateKey())
encryptedData = encrypt(data, keys, 0)
print encryptedData.tobytes()
decryptedData = encrypt(encryptedData, keys, 1)
print(decryptedData.tobytes())
s = hash_function(data, keys, 0)
print s
m = hash_function(data, keys, 0)
print m

Ly��y���U$�:n��8ɋ9�\aQ�%��n�Diem!
﻿Hello, my dear friend! Carpe Diem!
64
2221641530747876647
64
16607691975748002120
